In [33]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense, Conv2DTranspose, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# here we need only the images and not the target
(x_train, _), (x_test, _) = mnist.load_data()

In [3]:
x_train.shape,x_test.shape

In [4]:
plt.figure(figsize=(2,2))
plt.imshow(x_train[1], cmap="binary")

<h1>lets add some noise to this images</h1>

In [5]:
# first lets normelise
x_train = x_train / 255
x_test = x_test / 255

In [6]:
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [7]:
# adding random noise
NOISE_PROPORTION = 0.2
x_train_noisy = x_train + NOISE_PROPORTION * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + NOISE_PROPORTION * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

In [8]:
# we may have values in noisy images bigger than 1 so we need to clip them
x_train_noisy = np.clip(x_train_noisy, 0, 1)
x_test_noisy = np.clip(x_test_noisy, 0, 1)

In [9]:
plt.figure(figsize=(4,4))
ax = plt.subplot(1,2,1)
plt.imshow(x_train_noisy[1], cmap="binary")
ax = plt.subplot(1,2,2)
plt.imshow(x_train[1], cmap="binary")

<h1>the model (autoencoder)</h1>

In [27]:
model = Sequential()
# encoder
model.add(Input(shape=(28,28,1)))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
model.add(Conv2D(filters=8, kernel_size=(3,3), strides=1, activation='relu', padding='same'))
# decoder
model.add(Conv2DTranspose(filters=16, kernel_size=(3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2DTranspose(filters=32, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
model.add(Conv2DTranspose(filters=1, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
# model.add(Activation("sigmoid"))

In [28]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

In [29]:
model.fit(x_train_noisy, x_train, batch_size=128, epochs=50, validation_data=(x_test_noisy, x_test))

In [30]:
# lets try it on test images
no_noise_img = model.predict(x_test_noisy)

In [31]:
# lets see the images
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28), cmap="binary")
    
    # display reconstructed (after noise removed) image
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(no_noise_img[i].reshape(28, 28), cmap="binary")

plt.show()

<h1>lets try another architecture</h1>

In [39]:
model2 = Sequential()
# encoder
model2.add(Input(shape=(28,28,1)))
model2.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(filters=8, kernel_size=(3,3), strides=1, activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))
# decoder
model2.add(Conv2D(filters=8, kernel_size=(3,3), strides=1, activation='relu', padding='same'))
model2.add(UpSampling2D((2,2)))
model2.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'))
model2.add(UpSampling2D((2,2)))
model2.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model2.add(UpSampling2D((2,2)))
model2.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same'))

In [40]:
model2.compile(optimizer="adam", loss="mean_squared_error")
model2.summary()

In [41]:
model2.fit(x_train_noisy, x_train, batch_size=128, epochs=50, validation_data=(x_test_noisy, x_test))

In [42]:
# lets try it on test images
no_noise_img2 = model2.predict(x_test_noisy)

In [43]:
# lets see the images for the 2nd model
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28), cmap="binary")
    
    # display reconstructed (after noise removed) image
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(no_noise_img2[i].reshape(28, 28), cmap="binary")

plt.show()

In [44]:
# lets see the images
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28), cmap="binary")
    
    # display reconstructed (after noise removed) image
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(no_noise_img[i].reshape(28, 28), cmap="binary")

plt.show()